In [1]:
import numpy as np
import pandas as pd
import pydicom as dicom
import matplotlib.pylab as plt
import glob
import SimpleITK as sitk
import re
import random
random.seed(10)

import os
%env SITK_SHOW_COMMAND 'C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
dicom.config.convert_wrong_length_to_UN = True

env: SITK_SHOW_COMMAND='C:\ProgramData\slicer.org\Slicer 5.4.0\Slicer'


In [2]:
master_data = pd.read_csv('F:/U-CAN-Lymfom_A/Selected_for_UCAN_project/FinalSelected_exams_from_U-CAN-Lymfom_A_MasterDataset.csv')
print(master_data.shape)
master_data.head()

(1349, 33)


,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
0,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr134758508773-20191212\CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr134758508773-20191212\CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,ASPTCTX0001_npr134758508773-20191212,CT-20191212-112330-3.7-WB_FDG-CT-VENFAS-WB-Venfas-3-ax-3.000000mm_JK_,1.2.840.113619.2.5.42235195.12863.1576135318.405,npr134758508773,74.0,'72.0','1.64',512,512,442,0.976562,0.976562,3.00,"(512, 512, 442)","(0.976562, 0.976562, 3.0)","[-237.304, -233.886, 14.434]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842664208.103.1576074304.305.5240.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20191212','PET FDG WB 1 h','WB Venfas 3 ax','3.7 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI'
1,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr134758508773-20191212\PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr134758508773-20191212\PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,ASPTCTX0001_npr134758508773-20191212,PT-20191212-110507-3.7-WB_FDG-CT-VENFAS-QCFX-S-400-2.7900mm_JK_,1.2.840.113619.2.405.3842664208.1576145387.106224,npr134758508773,74.0,'72.0','1.64',256,256,395,1.953125,1.953125,2.79,"(256, 256, 395)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -1082.9699707031]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842664208.103.1576074304.305.5240.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20191212','PET FDG WB 1 h','QCFX-S 400','3.7 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI'
2,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr190483255078-20210203\CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr190483255078-20210203\CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,ASPTCTX0001_npr190483255078-20210203,CT-20210203-104152-3.12-WB_FDG-VENFAS--THORAX-INANDAD-WB-Venfas-3-ax-3.000000mm_JK_,1.2.840.113619.2.5.91447937.12518.1612342663.440,npr190483255078,75.0,'88.0','1.78',512,512,354,0.771484,0.771484,3.00,"(512, 512, 354)","(0.771484, 0.771484, 3.0)","[-197.885, -202.9, 4.309]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.582.1612334019.8.6305.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20210203','PET FDG WB 1 h','WB Venfas 3 ax','3.12 WB_FDG VENFAS + THORAX INANDAD',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI'
3,F:\U-CAN-Lymfom_A\Raw_DCM_transf_date_20221205__06__n86_90GB\ASPTCTX0001_npr190483255078-20210203\PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,U-CAN-Lymfom_A,Raw_DCM_transf_date_20221205__06__n86_90GB,ASPTCTX0001_npr190483255078-20210203\PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,ASPTCTX0001_npr190483255078-20210203,PT-20210203-102515-3.12-WB_FDG-VENFAS--THORAX-INANDAD-QCFX-S-400-M.Free-2.8000mm_JK_,1.2.840.113619.2.405.3842663696.1612344628.347570,npr190483255078,75.0,'88.0','1.78',256,256,314,1.953125,1.953125,2.80,"(256, 256, 314)","(1.953125, 1.953125, 2.8)","[-249.02343750000, -249.02343750000, -873.62500000000]","

In [4]:
set(master_data.source_directory.to_list())

{'Raw_DCM_transf_date_20221205__06__n86_90GB',
 'Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc',
 'Raw_DCM_transf_date_20221215__17_n199_424GBondisc',
 'Raw_DCM_transf_date_20221227_until_kl1215_n50_100GB'}

In [5]:
test_data = master_data[master_data['source_directory']=='Raw_DCM_transf_date_20221205__06__n86_90GB']
print(test_data.shape)

(50, 33)


In [13]:
def checkDistortedImg(vol_img,ptype='mean',angle=90):
    projection = {'sum': sitk.SumProjection,
                'mean':  sitk.MeanProjection,
                'std': sitk.StandardDeviationProjection,
                'min': sitk.MinimumProjection,
                'max': sitk.MaximumProjection}
    paxis = 0

    rotation_axis = [0,0,1]
    rotation_angles = np.linspace(-1/2*np.pi, 1/2*np.pi, int(180.0/angle)) #15.0 degree 

    rotation_center = vol_img.TransformContinuousIndexToPhysicalPoint([(index-1)/2.0 for index in vol_img.GetSize()])

    rotation_transform = sitk.VersorRigid3DTransform()
    rotation_transform.SetCenter(rotation_center)

    #Compute bounding box of rotating volume and the resampling grid structure
    image_indexes = list(zip([0,0,0], [sz-1 for sz in vol_img.GetSize()]))
    image_bounds = []
    for i in image_indexes[0]:
        for j in image_indexes[1]:
            for k in image_indexes[2]:
                image_bounds.append(vol_img.TransformIndexToPhysicalPoint([i,j,k]))

    all_points = []
    for angle in rotation_angles:
        rotation_transform.SetRotation(rotation_axis, angle)    
        all_points.extend([rotation_transform.TransformPoint(pnt) for pnt in image_bounds])
        
    all_points = np.array(all_points)
    min_bounds = all_points.min(0)
    max_bounds = all_points.max(0)

    #resampling grid will be isotropic so no matter which direction we project to
    #the images we save will always be isotropic (required for vol_img formats that 
    #assume isotropy - jpg,png,tiff...)

    new_spc = [np.min(vol_img.GetSpacing())]*3
    new_sz = [int(sz/spc + 0.5) for spc,sz in zip(new_spc, max_bounds-min_bounds)]

    for angle in rotation_angles:
        rotation_transform.SetRotation(rotation_axis, angle) 
        resampled_image = sitk.Resample(image1=vol_img,
                                        size=new_sz,
                                        transform=rotation_transform,
                                        interpolator=sitk.sitkLinear,
                                        outputOrigin=min_bounds,
                                        outputSpacing=new_spc,
                                        outputDirection = [1,0,0,0,1,0,0,0,1],
                                        defaultPixelValue =  -20, #HU unit for air in CT, possibly set to 0 in other cases
                                        outputPixelType = vol_img.GetPixelID())
        proj_image = projection[ptype](resampled_image, paxis)
        extract_size = list(proj_image.GetSize())
        extract_size[paxis]=0
        sitk.Extract(proj_image, extract_size)

In [20]:
exception_lst = []
for index, row in master_data.iterrows():

    #vol_img = sitk.ReadImage(row['directory'])
    
    #print(row['directory'])

    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(row['directory'])
    reader.SetFileNames(dicom_names)
    vol_img = reader.Execute()

    #name_suffix = r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CTMean&PETMax_Projections\\' + \
                  #str(row['patient_directory']) + str(row['modality'])
    #if row['modality']=="PT":
    #    ptype = 'max'
    #else:
    #    ptype = 'mean'

    try:
        checkDistortedImg(vol_img)#,img_n=name_suffix)
    except:
        exception_lst.append(row['directory'])

    #if index>10:
    #    break

exception_lst \

['F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr258071002437-20160805\\CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm',

 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr293454605342-20151210\\CT-20151210-093545-6.6-WB-F18-3MIN--CT-VENFAS-HELKROPP-VENFAS-AX-1-PCT-1.250000mm_JK_',
 
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr409683367521-20171127\\CT-20171127-133803-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm']

In [21]:
print(len(exception_lst)) 
print(exception_lst[:3])

77
['F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr258071002437-20160805\\CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm', 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr293454605342-20151210\\CT-20151210-093545-6.6-WB-F18-3MIN--CT-VENFAS-HELKROPP-VENFAS-AX-1-PCT-1.250000mm_JK_', 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr409683367521-20171127\\CT-20171127-133803-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm']


In [ ]:
with open(r'F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\DistortedImageTest\exception_lst.txt', 'w') as fp:
    for item in exception_lst:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')

In [27]:

# opening the file in read mode 
f = open(r"F:\U-CAN-Lymfom_A\Selected_for_UCAN_project\Selected_CT&PET_MaxProjections\exception_lst.txt", "r") 
  
# reading the file 
old_exceptions =  f.read()
old_exception_lst = old_exceptions.split("\n") 
  
# printing the data 
print(len(old_exception_lst)) 
print(old_exception_lst[:3]) 
f.close() 

86
['F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr258071002437-20160805\\CT-20160805-131025-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm', 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr293454605342-20151210\\CT-20151210-093545-6.6-WB-F18-3MIN--CT-VENFAS-HELKROPP-VENFAS-AX-1-PCT-1.250000mm_JK_', 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221205__06__n86_90GB\\ASPTCTX0001_npr409683367521-20171127\\CT-20171127-133803-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm']


In [30]:
check1 = list(set(old_exception_lst).difference(set(exception_lst)))
check1

['',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221215__17_n199_424GBondisc\\ASPTCTX0001_npr600444694366-20151215\\CT-20151215-115653-6.3-UAS-WB-FDG-3D-CT-THORAX-BUK-VENFAS-20141209-WB-VEN-AX-0.976562mm',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221215__17_n199_424GBondisc\\ASPTCTX0001_npr834522932360-20170112\\CT-20170112-152413-6.41-FDG--THORAX-WB-Venfas-1-ax-1.367188mm',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\\ASPTCTX0001_npr884176295970-20160921\\CT-20160921-143156-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221211__14_n574_1.2TBondisc\\ASPTCTX0001_npr334059381063-20161020\\CT-20161020-122530-6.3-UAS-WB-FDG-3D-CT-VENFAS-WB-VEN-AX-0.976562mm',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221215__17_n199_424GBondisc\\ASPTCTX0001_npr612146277420-20161220\\CT-20161220-142442-6.17-FDG--DT-WB-Venfas-1-ax-0.828125mm',
 'F:\\U-CAN-Lymfom_A\\Raw_DCM_transf_date_20221215__17_n199_424GBondisc\\ASPTCTX0001_n

In [35]:
master_data[master_data.directory.isin(check1)]

,directory,root_directory,source_directory,patient_info,patient_directory,PET-CT_info,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model
